# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.3 MB/s eta 0:00:00


In [ ]:
!mkdir /content/dataset/
!mkdir -p /content/dataset/train2C/
!mkdir -p /content/dataset/val2C/
!tar -xvzf /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train.tgz -C /content/dataset/train2C/
!tar -xvzf /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val.tgz -C /content/dataset/val2C/

Streaming output truncated to the last 5000 lines.
train/nevus/nev01090.jpg
train/nevus/nev03737.jpg
train/nevus/nev06203.jpg
train/nevus/nev01992.jpg
train/nevus/nev04542.jpg
train/nevus/nev04173.jpg
train/nevus/nev07309.jpg
train/nevus/nev01549.jpg
train/nevus/nev03977.jpg
train/nevus/nev00101.jpg
train/nevus/nev00499.jpg
train/nevus/nev00232.jpg
train/nevus/nev03062.jpg
train/nevus/nev03100.jpg
train/nevus/nev04661.jpg
train/nevus/nev04998.jpg
train/nevus/nev03432.jpg
train/nevus/nev07724.jpg
train/nevus/nev02586.jpg
train/nevus/nev07550.jpg
train/nevus/nev06346.jpg
train/nevus/nev05863.jpg
train/nevus/nev00483.jpg
train/nevus/nev00460.jpg
train/nevus/nev03889.jpg
train/nevus/nev07257.jpg
train/nevus/nev02541.jpg
train/nevus/nev05323.jpg
train/nevus/nev01568.jpg
train/nevus/nev06884.jpg
train/nevus/nev03661.jpg
train/nevus/nev01681.jpg
train/nevus/nev07438.jpg
train/nevus/nev00646.jpg
train/nevus/nev06119.jpg
train/nevus/nev06064.jpg
train/nevus/nev03147.jpg
train/nevus/nev00622.jpg

# Initialization

In [ ]:
%cd /content/drive/MyDrive/MaiaSpain/CAD/project/

/content/drive/.shortcut-targets-by-id/1fTsBspXCDEVY7q8PyaNkRq-51xfzOyIV/MaiaSpain/CAD/project


In [ ]:
import os
import numpy as np
from torch.utils.data import WeightedRandomSampler
from monai.transforms import (
    Compose, Rand2DElasticd, RandRotate90d, RandFlipd, RandAffined, ScaleIntensityd,
    RandCoarseShuffled, EnsureTyped, LoadImaged, Resized, ToTensord, NormalizeIntensityd
)
from monai.data import PersistentDataset,Dataset,DataLoader
from monai.losses import FocalLoss
from PIL import Image
import torch
from torch import nn, tensor
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, cohen_kappa_score
from tqdm import tqdm
from torchvision import models
from shutil import rmtree
import matplotlib.pyplot as plt

# Dataloader

In [ ]:
cls_to_idx = {'nevus':0,'others':1}
idx_to_cls = {0:'nevus',1:'others'}

def get_weights(data_path):
  classes = os.listdir(data_path)
  num_samples = {cls: len(os.listdir(os.path.join(data_path, cls))) for cls in classes}
  weights = {cls: (1 / num_samples[cls]) / sum((1 / v for v in num_samples.values())) for cls in classes}
  return weights

def get_data(data_path, cls_weights):
  data = []
  sample_weights = []
  classes = os.listdir(data_path)
  for cls in classes:
      cls_path = os.path.join(data_path, cls)
      for image_name in os.listdir(cls_path):
          image_path = os.path.join(cls_path, image_name)
          data.append({'image':image_path, 'label':cls_to_idx[cls]})
          sample_weights.append(cls_weights[cls])
  return data, sample_weights

In [ ]:
base_transforms = [
    LoadImaged('image',ensure_channel_first=True),
    Resized('image',(224, 224)),
    NormalizeIntensityd('image'),
]

augmentations = [
    ScaleIntensityd('image',0,1),
    Rand2DElasticd('image',prob=0.5, spacing=56, magnitude_range=(1, 4), padding_mode="zeros"),
    RandRotate90d('image',prob=0.5, spatial_axes=[0, 1]),
    RandFlipd('image',prob=0.5, spatial_axis=0),
    RandFlipd('image',prob=0.5, spatial_axis=1),
    RandAffined('image',prob=0.5,rotate_range=(0.75, 0.75),shear_range=(0.1, 0.1),padding_mode="zeros",),
    # RandCoarseShuffled('image',holes=1, spatial_size=10, max_holes=5, max_spatial_size=30, prob=0.5),
    NormalizeIntensityd('image'),
]

train_transform = Compose(base_transforms + augmentations)
val_transform = Compose(base_transforms)

rmtree('/content/cache/',True)

cls_weights = get_weights("/content/dataset/train2C/train")
train_data, train_weights = get_data("/content/dataset/train2C/train", cls_weights)
val_data, val_weights = get_data("/content/dataset/val2C/val", cls_weights)

train_dataset = PersistentDataset(data=train_data, transform=train_transform, cache_dir='/content/cache/train')
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))
train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, num_workers=os.cpu_count(), pin_memory=True)
val_dataset = PersistentDataset(data=val_data, transform=val_transform, cache_dir='/content/cache/val')
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
# train_loader.dataset.transform.transforms.append(ScaleIntensityd('image',0,1))
# plt.figure(figsize=(18,5))
# batch = next(iter(train_loader))
# for i in range(16):
#   image, label = batch['image'][i], batch['label'][i]
#   plt.subplot(2,8,i+1)
#   plt.imshow(image.permute(1,2,0).numpy())
#   plt.title(idx_to_cls[label.item()])
#   plt.axis('off')
# plt.tight_layout()
# plt.show()

# Training and Evaluation

In [ ]:
def calculate_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    f1 = f1_score(labels, preds, average="macro")
    kappa = cohen_kappa_score(labels, preds)
    return accuracy, precision, recall, f1, kappa

def train_and_evaluate_model(model, train_loader, val_loader, device, epochs=30, patience=20, min_delta=0.0001):
    model = model.to(device)
    criterion = FocalLoss(to_onehot_y=True, use_softmax=True)
    # optimizer = Adam(model.parameters(), lr=1e-3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.2)
    best_f1 = 0
    save_dir = "models_new/"
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch['image'].to(device), batch['label'].to(device)
                outputs = model(images).logits
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        accuracy, precision, recall, f1, kappa = calculate_metrics(all_labels, all_preds)
        cm = confusion_matrix(all_labels, all_preds)
        print(f"Validation Metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Kappa: {kappa:.4f}")
        print(f"Confusion Matrix:\n{cm}")

        if f1 > best_f1 + min_delta:
            best_f1 = f1
            model_path = os.path.join(save_dir, f"best_model_epoch_{epoch+1}.pth")
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        else:
            patience -= 1
            if patience <= 0:
                print("Early stopping!")
                break

# Experiments

In [ ]:
from transformers import DeiTForImageClassification, DeiTFeatureExtractor

model_name = "facebook/deit-base-distilled-patch16-224"
model = DeiTForImageClassification.from_pretrained(model_name, num_labels=len(cls_to_idx.keys()))
feature_extractor = DeiTFeatureExtractor.from_pretrained(model_name)

# model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)
# num_ftrs = model.classifier[1].in_features
# model.classifier[1] = nn.Linear(num_ftrs, len(cls_to_idx.keys()))

train_and_evaluate_model(
    model, train_loader, val_loader, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=100
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/349M [00:00<?, ?B/s]

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

Epoch 27/100:   0%|          | 0/238 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

Epoch 27/100, Training Loss: 0.0375



/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-

Validation Metrics - Accuracy: 0.8359, Precision: 0.8364, Recall: 0.8355, F1: 0.8357, Kappa: 0.6714
Confusion Matrix:
[[1655  276]
 [ 347 1518]]
Model saved to models_new/best_model_epoch_27.pth


Epoch 28/100:   0%|          | 0/238 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

In [ ]:
from transformers import DeiTForImageClassification, DeiTFeatureExtractor, DeiTConfig

config = DeiTConfig.from_pretrained("facebook/deit-base-distilled-patch16-224")
config.num_labels = 2
config.output_hidden_states = True
model = DeiTForImageClassification.from_pretrained("facebook/deit-base-distilled-patch16-224", config=config)

import torch
outputs = model(torch.randn(1,3,224,224))
pred = outputs.logits
distill_token = outputs.hidden_states[-1][:, -1, :]


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 2]) torch.Size([1, 768])


In [ ]:
print(pred.shape, distill_token.shape)

torch.Size([1, 2]) torch.Size([1, 768])


In [ ]:
distill_token.shape

google_token = None

distill_loss = distill_loss(distill_token, google_token)

focal_loss + distill_loss

torch.Size([1, 768])